$\begin{align}
	&u_t + c(x)u_x = 0, \text{ em } [0,5] \times [0,6.4], \\
	&u(x,0) = f(x), \\~\\
	&c(x) = 1/5+\sin(x-1)^2, \\
	&f(x) = e^{-100(x-1)^2}
\end{align}$

In [ ]:
using Pkg; Pkg.activate(".")

In [ ]:
using JLD; D = load("Upwind_Dados_grad.jld")["D"]; xt_D = D[1]; u_D = D[2]
length(u_D)

In [ ]:
using NeuralPDE, Lux, Random, Optimization, OptimizationOptimisers, ModelingToolkit, Zygote, Plots, LinearAlgebra
import ModelingToolkit: Interval
import Optimization: OptimizationFunction, OptimizationProblem, solve
Random.seed!(0);

In [ ]:
xmin = 0; xmax = 5; tmin = 0; tmax = 6.4
c(x) = 1/5+sin(x-1)^2
f(x) = exp(-100(x-1)^2)
aux(x,t) = ((t-2.0491*x) <= 1.4279) * ((t-2.0194*x) >= -4.9414)
u_exata(x,t) = exp(-100*(atan((1/sqrt(6))*tan(atan(sqrt(6)*tan(x-1))-(sqrt(6)/5)*t)))^2);

In [ ]:
Neurons = 1
Layers = 1

N_D = length(u_D)
N_I = 10;

In [ ]:
@parameters x t
@variables u(..)
Dt = Differential(t)
Dx = Differential(x)

PDE = [Dt(u(x,t)) + c(x)*Dx(u(x,t)) ~ 0]
IC = [u(x, 0) ~ f(x)]

Ω = [x ∈ Interval(xmin, xmax), t ∈ Interval(tmin, tmax)];

In [ ]:
Lux_NN = Chain(    
	Dense(2, Neurons, Lux.tanh; 
	init_weight = Lux.glorot_uniform, init_bias = Lux.glorot_uniform),

    [Dense(Neurons, Neurons, Lux.tanh; 
	init_weight = Lux.glorot_uniform, init_bias = Lux.glorot_uniform) for i in 1:1:Layers],

    Dense(Neurons, 1; 
	init_weight = Lux.glorot_uniform, init_bias = Lux.glorot_uniform))

global Weight_NN = Lux.setup(Random.default_rng(0), Lux_NN)[1];

In [ ]:
Weight_NN

In [ ]:
Strategy = FixedStochasticTraining(N_D, N_I) 

Discretization = PhysicsInformedNN(Lux_NN, Strategy, init_params = Weight_NN)

@named PDE_System = PDESystem(PDE, IC, Ω, [x, t], u(x, t))

Problem_NeuralPDE = symbolic_discretize(PDE_System, Discretization)

Weight_NN = Problem_NeuralPDE.flat_init_params

Strategy.sets[1] = xt_D'

pde_loss_functions = Problem_NeuralPDE.loss_functions.pde_loss_functions
bc_loss_functions = Problem_NeuralPDE.loss_functions.bc_loss_functions
Neural_Network(x,t,θ) = Problem_NeuralPDE.phi([x,t],θ)[1];

In [ ]:
function loss_fit(θ, p)
    return sum(abs2, Neural_Network(xt_D[i,1],xt_D[i,2],θ) - u_D[i] for i = 1:length(u_D))/length(u_D) 
end
function callback_fit(p, l)
	if p.iter%100 == 0
		println("iteration: ", p.iter)
		println("loss_fit: ", l)
	end
	return false
end

OptimizationFunction_fit = OptimizationFunction(loss_fit, AutoZygote())
Epoch_fit = 150
OptimizationProblem_fit = OptimizationProblem(OptimizationFunction_fit, Weight_NN) 
result_fit = solve(OptimizationProblem_fit, ADAM(10^-3); callback = callback_fit, maxiters = Epoch_fit)
Weight_NN = result_fit.u;

In [ ]:
Weight_NN

In [ ]:
function loss_fit_PINN(θ, α)
	fit = sum(abs2, Neural_Network(xt_D[i,1],xt_D[i,2],θ) - u_D[i] for i = 1:length(u_D))/length(u_D)
	PINN = sum(map(l -> l(θ), [pde_loss_functions; bc_loss_functions]))
    return α*fit + (1-α)*PINN
end
function callback_fit_PINN(P, l)
	if P.iter%100 == 0
		α = OptimizationProblem_fit_PINN.p
		println("α: ", α)
		println("iteration: ", P.iter)
		println("loss_fit_PINN: ", l)
		println("loss_fit: ", α*loss_fit(P.u, nothing))
		println("loss_PDE: ", (1-α)*map(l_ -> l_(P.u), pde_loss_functions)[1])
		println("loss_IC: ", (1-α)*map(l_ -> l_(P.u), bc_loss_functions)[1])
	end
	return false
end

OptimizationFunction_fit_PINN = OptimizationFunction(loss_fit_PINN, AutoZygote())
α = [0.8, 0.6, 0.4, 0.2]
Epoch_fit_PINN = [175, 175, 175, 175]
for i = 1:length(α)
    global OptimizationProblem_fit_PINN = OptimizationProblem(OptimizationFunction_fit_PINN, Weight_NN, α[i])
    result_fit_PINN = solve(OptimizationProblem_fit_PINN, ADAM(10^-3); callback = callback_fit_PINN, maxiters = Epoch_fit_PINN[i])
    Weight_NN = result_fit_PINN.u
end

In [ ]:
Weight_NN

In [ ]:
function loss_PINN(θ, p)
    return sum(map(l -> l(θ), [pde_loss_functions; bc_loss_functions]))
end
function callback_PINN(p, l)
	if p.iter%100 == 0
		println("iteration: ", p.iter)
		println("loss_PINN: ", l)
		println("loss_PDE: ", map(l_ -> l_(p.u), pde_loss_functions)[1])
		println("loss_IC: ", map(l_ -> l_(p.u), bc_loss_functions)[1])
	end
	return false
end

OptimizationFunction_PINN = OptimizationFunction(loss_PINN, AutoZygote())
Epoch_PINN = 150
OptimizationProblem_PINN = OptimizationProblem(OptimizationFunction_PINN, Weight_NN) 
result_PINN = solve(OptimizationProblem_PINN, ADAM(10^-3); callback = callback_PINN, maxiters = Epoch_PINN)
Weight_NN = result_PINN.u;

In [ ]:
Weight_NN

In [ ]:
PINN(x,t) = Neural_Network(x,t,Weight_NN);

In [ ]:
let
    Δ = 0.01; x = xmin:Δ:xmax; t = tmin:Δ:tmax
    u_PINN = [PINN(i, j) for j in t, i in x]
    u_EXATA = [u_exata(i, j)*aux(i, j) for j in t, i in x]
    u_erro = abs.(u_PINN - u_EXATA)
    h1 = heatmap(x, t, u_PINN, title="PINN")
    erro = norm(u_erro) 
    h2 = heatmap(x, t, u_erro, title="Erro")
    plot(h1, h2)
    #savefig("HeatMap.pdf")
    save("Resultado.jld", "Weight_NN", Weight_NN, "x", x, "t", t, "u_PINN", u_PINN)
end

In [ ]:
let
    Δ = 0.01; x = xmin:Δ:xmax
    u_PINN = [PINN(i, 0) for i in x]
    u_EXATA = [u_exata(i, 0)*aux(i, 0) for i in x]
    u_erro = abs.(u_PINN - u_EXATA)
    p1 = plot(x,[u_PINN u_EXATA], title=["PINN" "Exata"])
    erro = norm(u_erro)
    p2 = plot(x, u_erro, title="Erro")
    plot(p1, p2)
    #savefig("Plot.pdf")
end